# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = '/Users/xiehuizhu/Downloads/2020ML/Day_007/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df1_mean = df.groupby(['Cabin'])['Fare'].mean().reset_index()
df1_mode = df.groupby(['Cabin'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
df1_median = df.groupby(['Cabin'])['Fare'].median().reset_index()
df1_max = df.groupby(['Cabin'])['Fare'].max().reset_index()
df1_min = df.groupby(['Cabin'])['Fare'].min().reset_index()
df1_count = df['Cabin'].value_counts().reset_index().sort_index()
df1_count = pd.DataFrame(df1_count)
df1_count.columns = ['Cabin', 'Count']
temp = pd.merge(df1_mean, df1_mode, on = ['Cabin'], how = 'left')
temp = pd.merge(temp, df1_median, on = ['Cabin'], how = 'left')
temp = pd.merge(temp, df1_max, on = ['Cabin'], how = 'left')
temp = pd.merge(temp, df1_min, on = ['Cabin'], how = 'left')
temp = pd.merge(temp, df1_count, on = ['Cabin'], how = 'left')
temp.columns = ['Cabin', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min', 'Fare_count']
temp

,Cabin,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Fare_count
0,A10,40.125000,40.1250,40.12500,40.1250,40.1250,1
1,A14,52.000000,52.0000,52.00000,52.0000,52.0000,1
2,A16,39.600000,39.6000,39.60000,39.6000,39.6000,1
3,A19,26.000000,26.0000,26.00000,26.0000,26.0000,1
4,A20,56.929200,56.9292,56.92920,56.9292,56.9292,1
...,...,...,...,...,...,...,...
142,F33,11.333333,10.5000,10.50000,13.0000,10.5000,3
143,F38,7.750000,7.7500,7.75000,7.7500,7.7500,1
144,F4,39.000000,39.0000,39.00000,39.0000,39.0000,2
145,G6,13.581250,10.4625,13.58125,16.7000,10.4625,4


In [3]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df2_mean = df.groupby(['Embarked'])['Fare'].mean().reset_index()
df2_mode = df.groupby(['Embarked'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
df2_median = df.groupby(['Embarked'])['Fare'].median().reset_index()
df2_max = df.groupby(['Embarked'])['Fare'].max().reset_index()
df2_min = df.groupby(['Embarked'])['Fare'].min().reset_index()
df2_count = df['Embarked'].value_counts().reset_index().sort_index()
df2_count = pd.DataFrame(df2_count)
df2_count.columns = ['Embarked', 'Count']
temp2 = pd.merge(df2_mean, df2_mode, on = ['Embarked'], how = 'left')
temp2 = pd.merge(temp2, df2_median, on = ['Embarked'], how = 'left')
temp2 = pd.merge(temp2, df2_max, on = ['Embarked'], how = 'left')
temp2 = pd.merge(temp2, df2_min, on = ['Embarked'], how = 'left')
temp2 = pd.merge(temp2, df2_count, on = ['Embarked'], how = 'left')
temp2.columns = ['Embarked', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min', 'Fare_count']
temp2

,Embarked,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Fare_count
0,C,59.954144,7.2292,29.70,512.3292,4.0125,168
1,Q,13.276030,7.7500,7.75,90.0000,6.7500,77
2,S,27.079812,8.0500,13.00,263.0000,0.0000,644


In [4]:
df1 = pd.merge(df, temp, on = ['Cabin'], how = 'left')

In [5]:
df2 = pd.merge(df, temp2, on = ['Embarked'], how = 'left')

In [6]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df1.dtypes, df1.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df1 = df1[num_features]
df1 = df1.fillna(-1)
MMEncoder = MinMaxScaler()
df1.head()

11 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min', 'Fare_count']



,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Fare_count
0,3,22.0,1,0,7.2500,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0
1,1,38.0,1,0,71.2833,71.2833,71.2833,71.2833,71.2833,71.2833,1.0
2,3,26.0,0,0,7.9250,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0
3,1,35.0,1,0,53.1000,53.1000,53.1000,53.1000,53.1000,53.1000,2.0
4,3,35.0,0,0,8.0500,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0


In [7]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df2.dtypes, df2.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df2 = df2[num_features]
df2 = df2.fillna(-1)
MMEncoder = MinMaxScaler()
df2.head()

11 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min', 'Fare_count']



,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min,Fare_count
0,3,22.0,1,0,7.2500,27.079812,8.0500,13.0,263.0000,0.0000,644.0
1,1,38.0,1,0,71.2833,59.954144,7.2292,29.7,512.3292,4.0125,168.0
2,3,26.0,0,0,7.9250,27.079812,8.0500,13.0,263.0000,0.0000,644.0
3,1,35.0,1,0,53.1000,27.079812,8.0500,13.0,263.0000,0.0000,644.0
4,3,35.0,0,0,8.0500,27.079812,8.0500,13.0,263.0000,0.0000,644.0


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [8]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df2_minus = df2.drop(['Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min', 'Fare_count'], axis = 1)
train_X = MinMaxScaler().fit_transform(df2_minus)
estimator = LogisticRegression()
print(f'Logistic: {cross_val_score(estimator, train_X, train_Y, cv = 5).mean()}')

Logistic: 0.70041428661101


In [9]:
# 新特徵1 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MinMaxScaler().fit_transform(df1)
print(f'Logistic: {cross_val_score(estimator, train_X, train_Y, cv = 5).mean()}')

Logistic: 0.6970309459544286


In [10]:
# 新特徵2 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MinMaxScaler().fit_transform(df2)
print(f'Logistic: {cross_val_score(estimator, train_X, train_Y, cv = 5).mean()}')

Logistic: 0.6925679492812755


In [11]:
# 只保留 mean 1
df1_temp = df1.drop(['Fare_mode', 'Fare_min', 'Fare_count'], axis = 1)
train_X = MinMaxScaler().fit_transform(df1_temp)
print(f'Logistic: {cross_val_score(estimator, train_X, train_Y, cv = 5).mean()}')

Logistic: 0.702661477622246


In [12]:
# 只保留 mean 2
df2_temp = df2.drop(['Fare_mode', 'Fare_median', 'Fare_min', 'Fare_count', 'Fare_max'], axis = 1)
train_X = MinMaxScaler().fit_transform(df2_temp)
print(f'Logistic: {cross_val_score(estimator, train_X, train_Y, cv = 5).mean()}')

Logistic: 0.7026677546921098


mean 比較能代表做群聚編碼的類別變項。